In [ ]:
sc

In [ ]:
rawDataRDD=sc.textFile("audio_data/user_artist_data.txt")
#rawDataRDD.cache()

In [ ]:
print "Sampling the dataset to fit into laptop memory..."
weights = [.1, .9]
seed = 42
sample, someOtherJunk = rawDataRDD.randomSplit(weights, seed)
#sample.cache()

##check whether variables are in range for storing as int32.

In [ ]:
print sample.map(lambda x: float(x.split()[0])).stats()

In [ ]:
print sample.map(lambda x: float(x.split()[1])).stats()

In [ ]:
print sample.first()

##load the artist ID to name mappings.

In [ ]:
rawArtistRDD = sc.textFile("audio_data/artist_data.txt")

In [ ]:
def parseArtistIdNamePair(singlePair):
    splitPair = singlePair.rsplit('\t')
    # we should have two items in the list - id and name of the artist.
    if len(splitPair) != 2:
        return []
    else:
        try:
            return [(int(splitPair[0]), splitPair[1])]
        except:
            return []

In [ ]:
artistByID = dict(rawArtistRDD.flatMap(lambda x: parseArtistIdNamePair(x)).collect())
key = next(iter(artistByID))
value = artistByID[key]
print key, value
print artistByID[1000007]
artistByIDBroadcast = sc.broadcast(artistByID)
artistByIDBroadcast.value.get(1000007)

##load artist aliases.

In [ ]:
rawAliasRDD = sc.textFile("audio_data/artist_alias.txt")

In [ ]:
def parseArtistAlias(alias):
    splitPair = alias.rsplit('\t')
    # we should have two items in the list - id and name of the artist.
    if len(splitPair) != 2:
        #print singlePair
        return []
    else:
        try:
            return [(int(splitPair[0]), int(splitPair[1]))]
        except:
            return []

In [ ]:
artistAlias = rawAliasRDD.flatMap(lambda x: parseArtistAlias(x)).collectAsMap()
key = next(iter(artistAlias))
value = artistAlias[key]
print key, value
print artistByID[key]
print artistByID[value]

##convert all artist IDs to canonical form using aliases

In [ ]:
from pyspark.mllib import recommendation
from pyspark.mllib.recommendation import *
from pyspark.mllib.recommendation import ALS, Rating

In [ ]:
artistAliasBroadcast = sc.broadcast(artistAlias)

##build model

In [ ]:
def mapSingleObservation(x):
    userID, artistID, count = map(lambda lineItem: int(lineItem), x.split(' '))
    finalArtistID = artistAliasBroadcast.value.get(artistID)
    if finalArtistID is None:
        finalArtistID = artistID
    return Rating(userID, finalArtistID, count)

In [ ]:
trainData = sample.map(lambda x: mapSingleObservation(x))
trainData.cache()

In [ ]:
# rank = 10
#   The number of latent factors in the model, or equivalently, the number of columns
#   k in the user-feature and product-feature matrices. In nontrivial cases, this
#   is also their rank.
# iterations = 5
#   The number of iterations that the factorization runs. More iterations take more
#   time but may produce a better factorization.

model = ALS.trainImplicit(trainData, 10, 5, alpha=0.01)

##see if this model makes sense

In [ ]:
testUserID = 1000002
#grab all artists played by this test user

artistsForUser = (trainData
                  .filter(lambda observation: observation.user == testUserID)
                  .map(lambda observation: artistByIDBroadcast.value.get(observation.product))
                  .collect())
print artistsForUser

In [ ]:
#generate some recommendations and see if they make sense
# Got to use the model.call syntax here, because recommendProducts is not implemented
# In pySpark 1.3 :(
recommendationsForUser = \
    map(lambda observation: artistByID.get(observation.product), model.call("recommendProducts", testUserID, 30))

print recommendationsForUser